In [ ]:
!pip3 install fasttext

In [2]:
import fasttext
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.bin.gz

--2020-02-13 19:46:17--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4371554972 (4.1G) [application/octet-stream]
Saving to: ‘cc.hi.300.bin.gz’

cc.hi.300.bin.gz    100%[===================>]   4.07G  11.8MB/s    in 6m 52s  

2020-02-13 19:53:10 (10.1 MB/s) - ‘cc.hi.300.bin.gz’ saved [4371554972/4371554972]



In [0]:
!gunzip -k cc.hi.300.bin.gz

In [5]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [7]:
ft_model = fasttext.load_model('cc.hi.300.bin')

In [0]:
df = pd.read_csv('hi_3500 - Sheet1.csv', header=None, names=['text', 'sent'])

In [12]:
df.head()

,text,sent
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


In [13]:
df['sent'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [0]:
df['sent'] = df['sent'].replace(to_replace={'negative':0, 'positive':2, 'neutral':1})

In [0]:
def get_em(s):
    return ft_model.get_sentence_vector(s)

In [0]:
df['text'] = df['text'].str.replace('\n', '')

In [0]:
X, Y= df['text'].apply(get_em).values, df['sent'].values

In [0]:
X = np.array([np.array(x) for x in X])

In [0]:
Y = tf.keras.utils.to_categorical(Y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [0]:
tf.keras.backend.clear_session()

In [0]:
inputs = tf.keras.Input(shape=(300))
x = tf.keras.layers.Dense(units=32, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(units=32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(units=32, activation='relu')(x)
outputs = tf.keras.layers.Dense(units=3, activation='softmax')(x)

In [0]:
model = tf.keras.Model(inputs, outputs)

In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy()])

In [0]:
dir = str(datetime.datetime.now())
tb_callback = tf.keras.callbacks.TensorBoard('logs/'+dir, profile_batch=0)
checkpoint = tf.keras.callbacks.ModelCheckpoint('models/'+dir, monitor='val_accuracy', verbose=1, save_best_only=True)

In [25]:
model.fit(x_train, y_train, batch_size=16, initial_epoch=0, epochs=500, callbacks=[tb_callback, checkpoint], validation_data=(x_test, y_test))

Train on 7261 samples, validate on 1816 samples
Epoch 1/500
7232/7261 [============================>.] - ETA: 0s - loss: 0.9017 - accuracy: 0.5206 - categorical_accuracy: 0.5206
Epoch 00001: val_accuracy improved from -inf to 0.65584, saving model to models/2020-02-13 19:58:01.496519
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/2020-02-13 19:58:01.496519/assets
7261/7261 [==============================] - 3s 377us/sample - loss: 0.9005 - accuracy: 0.5213 - categorical_accuracy: 0.5213 - val_loss: 0.7276 - val_accuracy: 0.6558 - val_categorical_accuracy: 0.6558
Epoch 2/500
6928/7261 [===========================>..] - ETA: 0s - loss: 0.7285 - accuracy: 0.6516 - categorical_accuracy: 0.6516
Epoch 00002: val_accuracy improved from 0.65584 to 0.73844, saving model to models/2020-02-13 19:58:01.496519
INFO:tensorflow:Assets written to: models/2020-02-13 19:58:01.496519/assets
7261/7261 [============================